# 第9章: 事前学習済み言語モデル（BERT型）

本章では、BERT型の事前学習済みモデルを利用して、マスク単語の予測や文ベクトルの計算、評判分析器（ポジネガ分類器）の構築に取り組む。

In [8]:
!pip install transformers

In [9]:
import torch
import numpy as np
import torch.nn as nn
from tqdm import tqdm
from transformers import BertModel, BertTokenizer

In [10]:
model_name = "bert-base-uncased"
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

In [19]:
#tokenizer info
print(tokenizer.vocab_size)
print(tokenizer.model_max_length)
print(tokenizer.model_input_names)
print(tokenizer.cls_token)
print(tokenizer.sep_token)
print(tokenizer.pad_token)
print(tokenizer.unk_token)
print(tokenizer.mask_token)

30522
512
['input_ids', 'token_type_ids', 'attention_mask']
[CLS]
[SEP]
[PAD]
[UNK]
[MASK]


## 80. トークン化

"The movie was full of incomprehensibilities."という文をトークンに分解し、トークン列を表示せよ。

In [14]:
text_80 = "The movie was full of incomprehensibilities."

tokenized_text_80 = tokenizer(text_80)
print(tokenized_text_80)

{'input_ids': [101, 1996, 3185, 2001, 2440, 1997, 4297, 25377, 2890, 10222, 5332, 14680, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


## 81. マスクの予測

"The movie was full of [MASK]."の"[MASK]"を埋めるのに最も適切なトークンを求めよ。

In [26]:
special_tokens = ['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]']

for token in special_tokens:
    token_id = tokenizer.convert_tokens_to_ids(token)
    print(f"{token}: {token_id}")

[PAD]: 0
[UNK]: 100
[CLS]: 101
[SEP]: 102
[MASK]: 103


In [12]:
text_81 = "The movie was full of [MASK]."

inputs = tokenizer(text_81, return_tensors="pt")

mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

# 予測
with torch.no_grad():
    outputs = model(**inputs)
    print(outputs.last_hidden_state)


tensor([[[ 0.0803,  0.0524,  0.2160,  ..., -0.0484,  0.0396, -0.0810],
         [ 0.1399, -0.7367,  0.0800,  ..., -0.1352,  0.6658, -0.2520],
         [ 0.6495, -0.3994, -0.0092,  ...,  0.0797,  0.1946, -0.3643],
         ...,
         [-0.0195,  0.0931,  0.1026,  ...,  0.2136,  0.5399, -0.0682],
         [ 0.6450,  0.1955, -0.1401,  ...,  0.0967, -0.6562, -0.4201],
         [ 0.2711,  0.2581,  0.7230,  ...,  0.3409, -0.5675,  0.1125]]])


## 82. マスクのtop-k予測

"The movie was full of [MASK]."の"[MASK]"に埋めるのに適切なトークン上位10個と、その確率（尤度）を求めよ。

## 83. CLSトークンによる文ベクトル

以下の文の全ての組み合わせに対して、最終層の[CLS]トークンの埋め込みベクトルを用いてコサイン類似度を求めよ。

- "The movie was full of fun."
- "The movie was full of excitement."
- "The movie was full of crap."
- "The movie was full of rubbish."


## 84. 平均による文ベクトル

以下の文の全ての組み合わせに対して、最終層の埋め込みベクトルの平均を用いてコサイン類似度を求めよ。

- "The movie was full of fun."
- "The movie was full of excitement."
- "The movie was full of crap."
- "The movie was full of rubbish."

## 85. データセットの準備

[General Language Understanding Evaluation (GLUE)](https://gluebenchmark.com/) ベンチマークで配布されている[Stanford Sentiment Treebank (SST)](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip) から訓練セット（train.tsv）と開発セット（dev.tsv）のテキストと極性ラベルと読み込み、さらに全てのテキストはトークン列に変換せよ。

## 86. ミニバッチの作成

85で読み込んだ訓練データの一部（例えば冒頭の4事例）に対して、パディングなどの処理を行い、トークン列の長さを揃えてミニバッチを構成せよ。

## 87. ファインチューニング

訓練セットを用い、事前学習済みモデルを極性分析タスク向けにファインチューニングせよ。検証セット上でファインチューニングされたモデルの正解率を計測せよ。

## 88. 極性分析

問題87でファインチューニングされたモデルを用いて、以下の文の極性を予測せよ。

- "The movie was full of incomprehensibilities."
- "The movie was full of fun."
- "The movie was full of excitement."
- "The movie was full of crap."
- "The movie was full of rubbish."


## 89. アーキテクチャの変更

問題87とは異なるアーキテクチャ（例えば[CLS]トークンを用いるか、各トークンの最大値プーリングを用いるなど）の分類モデルを設計し、事前学習済みモデルを極性分析タスク向けにファインチューニングせよ。検証セット上でファインチューニングされたモデルの正解率を計測せよ。